In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from hmeasure import h_score
from sklearn.metrics import roc_curve, auc

In [2]:
import pandas as pd
from openpyxl import load_workbook

# 读取滞后两年ST工作表
workbook_2st = load_workbook('C:/Users/DuShuyi/Desktop/财务困境/company_st/滞后两年.xlsx')
sheet_2st = workbook_2st['滞后两年ST']

# 跳过第一行，假设第一行是标题行
data_rows_2st = sheet_2st.iter_rows(min_row=2, values_only=True)
rawdata_2st = pd.DataFrame(data_rows_2st, columns=[cell.value for cell in next(sheet_2st.iter_rows(min_row=1))])

# 读取滞后两年非ST工作表
workbook_2nst = load_workbook('C:/Users/DuShuyi/Desktop/财务困境/company_st/滞后两年.xlsx')
sheet_2nst = workbook_2nst['滞后两年非ST']

# 跳过第一行，假设第一行是标题行
data_rows_2nst = sheet_2nst.iter_rows(min_row=2, values_only=True)
rawdata_2nst = pd.DataFrame(data_rows_2nst, columns=[cell.value for cell in next(sheet_2nst.iter_rows(min_row=1))])

In [3]:
data_2st=rawdata_2st.iloc[:,3:]
data_2nst=rawdata_2nst.iloc[:,3:]

In [4]:
data_2st=np.array(data_2st)
data_2nst=np.array(data_2nst)
print(data_2st.shape,data_2nst.shape)

(955, 81) (35638, 81)


In [5]:
data_2st_X = data_2st[:,:-1]
data_2st_Y = data_2st[:,-1:]
data_2nst_X = data_2nst[:,:-1]
data_2nst_Y = data_2nst[:,-1:]

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
Normalized=MinMaxScaler()

print(data_2st_X .shape,data_2nst_X.shape )
data_merge_2X = np.vstack((data_2st_X,data_2nst_X))
print(data_merge_2X.shape)
data_normalize_2 = Normalized.fit_transform(data_merge_2X)
data_2st_X = data_normalize_2[:len(data_2st_X),:]
data_2nst_X = data_normalize_2[len(data_2st_X):,:]
print(data_2st_X .shape,data_2nst_X.shape)

(955, 80) (35638, 80)
(36593, 80)
(955, 80) (35638, 80)


In [7]:
x_train_2st, x_test_2st, y_train_2st, y_test_2st = train_test_split(data_2st_X, data_2st_Y, test_size=0.08, random_state=1)
print(x_train_2st.shape, x_test_2st.shape, y_train_2st.shape, y_test_2st.shape)
x_train_2nst, x_test_2nst, y_train_2nst, y_test_2nst = train_test_split(data_2nst_X, data_2nst_Y, test_size=0.08, random_state=1)
print(x_train_2nst.shape, x_test_2nst.shape, y_train_2nst.shape, y_test_2nst.shape)

(878, 80) (77, 80) (878, 1) (77, 1)
(32786, 80) (2852, 80) (32786, 1) (2852, 1)


In [8]:
x_train_2=np.vstack((x_train_2st,x_train_2nst))
x_test_2=np.vstack((x_test_2st,x_test_2nst))
y_train_2=np.vstack((y_train_2st,y_train_2nst))
y_test_2=np.vstack((y_test_2st,y_test_2nst))

In [9]:
x_train = x_train_2
y_train = y_train_2
x_test = x_test_2
y_test = y_test_2

In [10]:
def caculate_TNR_TPR(confusion_matrix):
    TP = confusion_matrix[1][1]
    TN = confusion_matrix[0][0]
    FP = confusion_matrix[0][1]
    FN = confusion_matrix[1][0]
    
    TNR = TN/(TN+FP)
    TPR = TP/(TP+FN)
    
    return TNR

In [12]:
from imblearn.ensemble import EasyEnsembleClassifier
from sklearn import metrics 
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.svm import NuSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression

xgb_model = EasyEnsembleClassifier(base_estimator=XGBClassifier(learning_rate = 0.002861,n_estimators = 76,max_depth = 18), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
xgb_model.fit(x_train, y_train)

rf_model = EasyEnsembleClassifier(base_estimator=RandomForestClassifier(n_estimators =10, max_depth=20, random_state=10), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
rf_model.fit(x_train, y_train)

ada_model = EasyEnsembleClassifier(base_estimator=AdaBoostClassifier(n_estimators=141,learning_rate = 0.1), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
ada_model.fit(x_train, y_train)

gbdt_model = EasyEnsembleClassifier(base_estimator=GradientBoostingClassifier(n_estimators=59, learning_rate=0.01, max_depth=9), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
gbdt_model.fit(x_train, y_train)

svm_model = EasyEnsembleClassifier(base_estimator=svm.NuSVC(kernel='rbf',nu=0.04687), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
svm_model.fit(x_train, y_train)

LR_model = EasyEnsembleClassifier(base_estimator=LogisticRegression(solver='saga', penalty = 'l1'), n_estimators=100, n_jobs=1,
            random_state=1, replacement=False, sampling_strategy='auto',
            verbose=0, warm_start=False)
LR_model.fit(x_train, y_train)

D:\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\anaconda3\lib\site-packages\sklearn\ensemble\_bagging.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1

D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\a

D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
D:\a

EasyEnsembleClassifier(base_estimator=LogisticRegression(penalty='l1',
                                                         solver='saga'),
                       n_estimators=100, n_jobs=1, random_state=1)

In [14]:
y_actual = y_test
y_pred_xgb = xgb_model.predict(x_test)
y_pred_rf = rf_model.predict(x_test)
y_pred_ada = ada_model.predict(x_test)
y_pred_gbdt = gbdt_model.predict(x_test)
y_pred_LR = LR_model.predict(x_test)
y_pred_svm = svm_model.predict(x_test)

In [15]:
y_pred = y_pred_xgb

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = xgb_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      2852
         1.0       0.16      0.77      0.26        77

    accuracy                           0.89      2929
   macro avg       0.58      0.83      0.60      2929
weighted avg       0.97      0.89      0.92      2929

[[2537  315]
 [  18   59]]
AUC: 0.8279
Accuracy: 0.8863
precision: 0.1578
f1: 0.2616
recall: 0.7662
TNR: 0.8896
h_measure: 0.4984
KS: 0.6716316642684105


In [17]:
y_pred = y_pred_ada

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = ada_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.88      0.93      2852
         1.0       0.15      0.75      0.25        77

    accuracy                           0.88      2929
   macro avg       0.57      0.82      0.59      2929
weighted avg       0.97      0.88      0.92      2929

[[2520  332]
 [  19   58]]
AUC: 0.8184
Accuracy: 0.8802
precision: 0.1487
f1: 0.2484
recall: 0.7532
TNR: 0.8836
h_measure: 0.4720
KS: 0.6656845959089998


In [16]:
y_pred = y_pred_rf

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = rf_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      2852
         1.0       0.16      0.74      0.26        77

    accuracy                           0.89      2929
   macro avg       0.58      0.82      0.60      2929
weighted avg       0.97      0.89      0.92      2929

[[2550  302]
 [  20   57]]
AUC: 0.8172
Accuracy: 0.8901
precision: 0.1588
f1: 0.2615
recall: 0.7403
TNR: 0.8941
h_measure: 0.4800
KS: 0.6432578641554798


In [18]:
y_pred = y_pred_gbdt

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = gbdt_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.89      0.94      2852
         1.0       0.15      0.75      0.25        77

    accuracy                           0.88      2929
   macro avg       0.57      0.82      0.59      2929
weighted avg       0.97      0.88      0.92      2929

[[2529  323]
 [  19   58]]
AUC: 0.8200
Accuracy: 0.8832
precision: 0.1522
f1: 0.2533
recall: 0.7532
TNR: 0.8867
h_measure: 0.4785
KS: 0.6572831095972751


In [19]:
y_pred = y_pred_svm

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = svm_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.81      0.89      2852
         1.0       0.10      0.81      0.18        77

    accuracy                           0.81      2929
   macro avg       0.55      0.81      0.54      2929
weighted avg       0.97      0.81      0.87      2929

[[2315  537]
 [  15   62]]
AUC: 0.8085
Accuracy: 0.8115
precision: 0.1035
f1: 0.1834
recall: 0.8052
TNR: 0.8117
h_measure: 0.3884
KS: 0.6302298683084098


In [20]:
y_pred = y_pred_LR

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵


y_predict_proba = LR_model.predict_proba(x_test)[:,1]
fpr,tpr,thresholds= roc_curve(y_actual,y_predict_proba)

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
precision = metrics.precision_score(y_actual, y_pred )  # 求准度
f1 = metrics.f1_score(y_actual, y_pred )  # 求f1
recall = metrics.recall_score(y_actual, y_pred  )   # 求recall
TNR = caculate_TNR_TPR(metrics.confusion_matrix(y_actual, y_pred ))
h_measure = h_score(y_actual, y_pred)
print("AUC: %.4f" % (auc))
print("Accuracy: %.4f" % (accuracy))
print("precision: %.4f" % (precision))
print("f1: %.4f" % (f1))
print("recall: %.4f" % (recall))
print("TNR: %.4f" % (TNR))
print("h_measure: %.4f" % (h_measure))
print ('KS:',max(tpr-fpr))

              precision    recall  f1-score   support

         0.0       0.99      0.60      0.74      2852
         1.0       0.05      0.73      0.09        77

    accuracy                           0.60      2929
   macro avg       0.52      0.66      0.42      2929
weighted avg       0.96      0.60      0.73      2929

[[1703 1149]
 [  21   56]]
AUC: 0.6622
Accuracy: 0.6005
precision: 0.0465
f1: 0.0874
recall: 0.7273
TNR: 0.5971
h_measure: 0.0903
KS: 0.4477923899382525


In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier

bbc = BalancedBaggingClassifier(base_estimator=RandomForestClassifier(),
                                n_estimators= 50,
                                replacement=False,
                                random_state=0)
bbc.fit(x_train, y_train) 
y_actual = y_test
y_pred = ee_model.predict(x_test)

print(metrics.classification_report(y_actual, y_pred ))  # 输出结果，精确度、召回率、f-1分数
print(metrics.confusion_matrix(y_actual, y_pred ))  # 混淆矩阵

auc = metrics.roc_auc_score(y_actual, y_pred )
accuracy = metrics.accuracy_score(y_actual, y_pred )  # 求精度
print("Accuracy: %.2f%%" % (accuracy * 100.0))